In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn

In [2]:
word_groups = [
    # About documents
    ['unreport', 'unreported', 'unreporting', 'unreports', 'report', ....., 'misreport',....., 'nonreport',...., 'underreport', 'underreports', 'underreported', 'underreporting'],
    ['unrecord, record, nonrecord',...],
    ['undocumented', 'undocument', 'undocumenting', 'undocuments'],
    ['misrepresent', 'misrepresented', 'misrepresenting', 'misreports'],
    ['register', 'registers', 'registered', 'registering', 'unregister', 'unregisters', 'unregistered', 'unregistering'],
    ['logbook', 'logged'],
    ['declaration', 'declare', 'declares', 'declared', 'declaring', 'underdeclaration', 'under-declaration'],
    ['label', 'mislabel', 'labeling', 'labeled', 'labels', 'mislabeling', 'mislabled', 'mislables'],
    ['evade',],
    ['disappear',....,'reappear',...,'appear',.... ],
    ['reveal',..., 'discover',..., 'vanish', ],
    ['endanger',..., 'protect',..., 'restrict',...],
    ['undisclose',..., 'disclose',.... ],
    ['ban',..., 'restrict',...],
    ['hide',..],
    ['bypass',],
    
    # About being false
    ['false', 'falsify', 'falsifies', 'falsifying', 'falsifies', 'falsified', 'falsification', 'fake', 'manipulated', 'manipulate', 'manipulates', 'manipulating', 'manipulation'],
    ['fraud', 'fraudulence', 'hoax'],
    # About amounts
    ['volume', 'quota', 'exceed', 'exceeds', 'exceeded', 'exceeding', 'inflation'],
    # Fish, Ocean, port 
    ['catch', 'bycatch', 'harvest', 'juvenile', 'invasive'],
    ['farmed', 'farming'], 
    ['ocean', 'sea', 'seas', 'water'],
    ['port'],
    ['transshipment'],
    ['landing', 'trade','import'],
    ['selective'],    
#    ['commercial', 'industrial'],
    # Crime
    ['offend', 'offended', 'offends', 'violate', 'violates', 'violated', 'violation'],
    ['regulation', 'regulate', 'regulated', 'regulates', 'regulating', 'regulations', 'regulates', unregulate,],
    ['alter', 'altering', 'alters'],
    ['exploit', 'exploited', 'exploiting', 'exploits'],
    ['impose', 'imposed', 'imposes', 'imposing'],
    ['detect', ..., 'prosecute',...]
    ['inspector', 'inspectors', 'inspect', 'inspects', 'inspected', 'inspecting','investigate', 'investigates', 'investigated','investigating','investigator'],
]

In [ ]:
 #, aggregate, , import, export, monitor, unload, offload overload load, spaw, doctor, undeclare, permit, threaten

In [3]:
import requests
from bs4 import BeautifulSoup

########################################################
#### This function scrap the contents in the url
#### input: url
#### output: content (type: str)
####
#### required libraries:
# import requests
# from bs4 import BeautifulSoup
########################################################

def scrape_content(url):
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the webpage
        soup = BeautifulSoup(response.text, 'html.parser')
#print(soup)
        # Extract the desired information
        # Example: Extracting all paragraphs from the webpage
        paragraphs = soup.find_all('p')

        content = ""
        # Print or process the extracted information
#        for paragraph in paragraphs:
#            content += " " + paragraph.text

###### The below is to only return a shorter content. For the full content, use the commented commands above
        for i in range(min(len(paragraphs), 10)):
### For the whole content:
#        for i in range(len(paragraphs)):
            content += " " + paragraphs[i].text
        return content
    else:
        print("Failed to retrieve content. Status code:", response.status_code)
        return None
    
scrape_content('https://www.msn.com/en-ca/lifestyle/other/experts-urge-people-to-fish-and-eat-crab-species-putting-entire-fishing-industry-at-risk-an-animal-of-unacceptable-intelligence/ar-BB1na7wc?ocid=BingNewsSearch')

''

In [4]:
import requests
from bs4 import BeautifulSoup

########################################################
#### This function scrap the titles, links, and dates of the first 20 articles from google news with the given query
#### input: query
#### output: the list of the title and link
####
#### required libraries:
# import requests
# from bs4 import BeautifulSoup
#
# Note: The dates are in the form of "1 month ago", "3 hours ago"
########################################################

def scrape_google_news(query):
    # Construct the Google News URL with the query
    url = f"https://www.google.com/search?q={query}&tbm=nws"
    print(url)
#    url = f"https://news.google.com/search?q={query}"

    # Send a GET request to the URL
    response = requests.get(url)

    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all the search result elements
    search_results = soup.find_all('div', class_='Gx5Zad fP1Qef xpd EtOod pkphOe')
    date_elements = soup.find_all('span', class_='r0bn4c rQMQod')


    # Extract the title and link of each search result
    scrap = []
    for i in range(min(len(search_results), 5)):
        title = search_results[i].find('a').text
        link = search_results[i].find('a')['href']
        #link = 'https://news.google.com' + link[1:]
        date = date_elements[i].text   
        scrap.append({'title': title, 'link': link, 'date': date})   
    return scrap

In [5]:
import pandas as pd

# read the excel file
excel_data = pd.read_excel('PIMS Sample Prompts.xlsx')

queries = []
for index, row in excel_data.iterrows():
    # Process each row
    queries.append(row['Prompt'])
    
queries = queries[:2]

In [6]:
### Test with differnet queries
query = input("Enter your search query: ")

# When we want to use the queries from the excel file:
#for query in queries:
#    top_results = scrape_google_news(query)
#    for index, result in enumerate(top_results, start=1):
#        print(f"{index}. {result['title']}")
#        link = "https://news.google.com" + result['link'][1:]
#        print(link)
#        print()
#        print(scrape_content(link))


top_results = scrape_google_news(query)

for index, result in enumerate(top_results, start=1):
    print(f"{index}. {result['title']}")
    link = result['link'][7:]
    print(link)
    print(result['date'])
    print(scrape_content(link))
    print()    

Enter your search query:  Vessel caught misreporting catch amount


https://www.google.com/search?q=Vessel caught misreporting catch amount&tbm=nws
1. Sea Shepherd GlobalSea Shepherd GlobalSea Shepherd Global stands at the forefront of the fight against Illegal, Unreported, and Unregulated (IUU) fishing, deploying innovative...2 weeks ago
https://www.seashepherdglobal.org/latest-news/combat-iuu-fishing/&sa=U&ved=2ahUKEwjkrqPF_e-GAxXPOjQIHcb2G1UQxfQBegQIBRAC&usg=AOvVaw3Cu8PkdOYu-fqd2ueoPEmj
2 weeks ago
Failed to retrieve content. Status code: 404
None

2. New rules tighten controls on EU…Environmental Justice FoundationEuropean Commission approves new rules that require stricter controls for landings by EU fishing vessels.1 month ago
https://ejfoundation.org/news-media/new-rules-tighten-controls-on-eu-vessels-to-prevent-misreporting&sa=U&ved=2ahUKEwjkrqPF_e-GAxXPOjQIHcb2G1UQxfQBegQICRAC&usg=AOvVaw2NwYoRjrYtpQI5mPXwe-dM
1 month ago
Failed to retrieve content. Status code: 404
None

3. N.S. boat captain, 2 companies fined $125K for fisheries violationsCBC